In [1]:
!pip install pyspark

In [1]:
import pandas as pd
import os
import glob
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DataEngineerAccelerator").getOrCreate()


In [14]:
product_df = spark.read.csv("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Sourced/SystemA/Product/date=20220606/Product.csv", header="True")
productcategory_df = spark.read.csv("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Sourced/SystemA/ProductCategory/date=20220606/ProductCategory.csv", header="True")
sales_df = spark.read.csv("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Sourced/SystemA/Sales/*", header="True")

def read_csv_to_df(file_path):
    df = spark.read.csv(file_path, header="True")
    return df

read_csv_to_df("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Sourced/SystemA/Product").printSchema()

NameError: name 'schema' is not defined

# Writing files to Cleansed

In [4]:

current_date_df = spark.range(1).select(to_timestamp(current_timestamp()).alias("current_date"))
current_date_df.show()

year_df = current_date_df.select(year("current_date").alias("submission_year"))
month_df = current_date_df.select(month("current_date").alias("submission_month"))
day_df = current_date_df.select(day("current_date").alias("submission_day"))
hour_df = current_date_df.select(hour("current_date").alias("submission_hour"))
minute_df = current_date_df.select(minute("current_date").alias("submission_minute"))
second_df = current_date_df.select(second("current_date").alias("submission_second"))

submission_year = year_df.first()["submission_year"]
submission_month = month_df.first()["submission_month"]
submission_day = day_df.first()["submission_day"]
submission_hour = hour_df.first()["submission_hour"]
submission_minute = minute_df.first()["submission_minute"]
submission_second = second_df.first()["submission_second"]

+--------------------+
|        current_date|
+--------------------+
|2023-10-18 10:22:...|
+--------------------+



In [5]:
def DataFeed(file_path):
    df = spark.read.csv(file_path, header="True")
    df = df.withColumn("sourcefile",input_file_name())
    df = df.withColumn("sourcefile", substring_index("sourcefile","/", -1))
    df = df.withColumn("sourcefile", split(col("sourcefile"), "\\.")[0]).select("sourcefile").distinct()
    df = df.withColumn("sourcefile", split(col("sourcefile"), "%")[0]).select("sourcefile").distinct()
    df = df.first()["sourcefile"]
    return df

data_feed = DataFeed("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Sourced/SystemA/Sales/date=20220607")
print(data_feed)

Sales


In [6]:
df = spark.read.csv("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Sourced/SystemA/Product", header="True")
df = df.withColumn("sourcefile",input_file_name())
df = df.withColumn("sourcefile", substring_index("sourcefile","/", -1))
df = df.withColumn("sourcefile", split(col("sourcefile"), "\\.")[0]).select("sourcefile").distinct()
df = df.withColumn("sourcefile", split(col("sourcefile"), "%")[0]).select("sourcefile").distinct()

df.show()
df = df.first()["sourcefile"]
print(df)

+----------+
|sourcefile|
+----------+
|   Product|
+----------+

Product


In [7]:
cleansed_path = f"Cleansed/DataFeed={data_feed}/schemaVersion=1/SubmissionYear={submission_year}/SubmissionMonth={submission_month}/SubmissionDay={submission_day}/SubmissionHour={submission_hour}/SubmissionMinute={submission_minute}/SubmissionSecond={submission_second}"
print(cleansed_path)

Cleansed/DataFeed=Sales/schemaVersion=1/SubmissionYear=2023/SubmissionMonth=10/SubmissionDay=18/SubmissionHour=10/SubmissionMinute=22/SubmissionSecond=26


# Column mapping

In [3]:
productcategory_mapping = {
    "ID" : ("ProductCategoryID", "string"),
    "Level1" : ("ProductCategoryName", "string"),
    "Level2" : ("ProductSubCategoryName", "string")
}

product_mapping = {
    "ProductID" : ("SourceProductID", "string"),
    "ModelName" : ("ProductModelName", "string"),
    "ProductName" : ("ProductName", "string"),
    "ProductColor" : ("ProductColor", "string"),
    "Size" : ("ProductSize", "string"),
    "ProductWeight" : ("ProductWeightKilograms", "double"),
    "UniqueProductNumber" : ("ProductUID", "string"),
    "ProductCategoryID" : ("ProductCategoryID", "string"),
    "StandardCost" : ("ProductCost", "double"),
    "ListPrice" : ("ProductListPrice", "double"),
    "ProductLine" : ("ProductLine", "string"),
    "Class" : ("ProductClass", "string"),
    "Style" : ("ProductStyle", "string"),
    "ProductMakeFlag" : ("ProductMakeFlag", "string"),
    "ProductFinishedGoodsFlag" : ("IsFinishedGood", "boolean"),
    "SellStartDate" : ("ProductSellingStartDate", "string"),
    "SellEndDate" : ("ProductSellingEndDate", "string"),
    "ProductID" : ("SourceProductID", "string"),
    "DiscontinuedDate" : ("ProductDiscontinuedDate", "string"),
    "ModifiedDate" : ("ModifiedDate", "string")
}

sales_mapping = {
    "SalesOrderID" : ("SalesOrderID", "string"),
    "SalesOrderDetailID" : ("SalesOrderLineID", "string"),
    "SalesOrderNumber" : ("SalesOrderNumber", "string"),
    "OrderDate" : ("SalesOrderDate", "string"),
    "ProductID" : ("ProductID", "string"),
    "SpecialOfferID" : ("SpecialOfferID", "string"),
    "SpecialOfferName" : ("SpecialOfferName", "string"),
    "SpecialOfferDiscountPct" : ("SpecialOfferDiscountPct", "string"),
    "SpecialOfferType" : ("SpecialOfferType", "string"),
    "SpecialOfferCategory" : ("SpecialOfferCategory", "string"),
    "SpecialOfferStartDate" : ("SpecialOfferStartDate", "string"),
    "SpecialOfferEndDate" : ("SpecialOfferEndDate", "string"),
    "OrderQty" : ("TotalUnits", "int"),
    "UnitPrice" : ("UnitPrice", "double"),
    "UnitPriceDiscount" : ("UnitPriceDiscountPercentage", "double"),
    "LineTotal" : ("TotalLineValue", "double")
}

In [27]:
from pyspark.sql.types import (StructType, StructField, StringType, IntegerType, DoubleType, BooleanType, LongType)

cleansed_product_category_schema = StructType([
    StructField("ProductCategoryID", StringType()),
    StructField("ProductCategoryName", StringType()),
    StructField("ProductSubCategoryName", StringType())
])    

product_schema = StructType([
    StructField("SourceProductID", StringType()),
    StructField("ProductModelName", StringType()),
    StructField("ProductName", StringType()),
    StructField("ProductColor", StringType()),
    StructField("ProductSize", StringType()),
    StructField("ProductWeightKilograms", DoubleType()),
    StructField("ProductUID", StringType()),
    StructField("ProductCategoryID", StringType()),
    StructField("ProductCost", DoubleType()),
    StructField("ProductListPrice", DoubleType()),
    StructField("ProductLine", StringType()),
    StructField("ProductClass", StringType()),
    StructField("ProductStyle", StringType()),
    StructField("ProductMakeFlag", StringType()),
    StructField("IsFinishedGood", BooleanType()),
    StructField("ProductSellingStartDate", StringType()),
    StructField("ProductSellingEndDate", StringType()),
    StructField("ProductDiscontinuedDate", StringType()),
    StructField("ModifiedDate", StringType())
])    

sales_schema = StructType([
    StructField("SalesOrderID", StringType()),
    StructField("SalesOrderID", StringType()),
    StructField("SalesOrderNumber", StringType()),
    StructField("SalesOrderDate", StringType()),
    StructField("ProductID", StringType()),
    StructField("SpecialOfferID", StringType()),
    StructField("SpecialOfferName", StringType()),
    StructField("SpecialOfferDiscountPct", DoubleType()),
    StructField("SpecialOfferType", StringType()),
    StructField("SpecialOfferCategory", StringType()),
    StructField("SpecialOfferStartDate", StringType()),
    StructField("SpecialOfferEndDate", StringType()),
    StructField("TotalUnits", IntegerType()),
    StructField("UnitPrice", DoubleType()),
    StructField("UnitPriceDiscountPercentage", DoubleType()),
    StructField("TotalLineValue", DoubleType())
])
    
    
schema = cleansed_product_category_schema
df = spark.createDataFrame(productcategory_df.rdd, schema=schema)
print("Number of Rows: ", df.count())

Py4JJavaError: An error occurred while calling o252.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 21.0 failed 1 times, most recent failure: Lost task 0.0 in stage 21.0 (TID 21) (host.docker.internal executor driver): java.net.SocketException: Connection reset
	at java.base/sun.nio.ch.NioSocketImpl.implRead(NioSocketImpl.java:318)
	at java.base/sun.nio.ch.NioSocketImpl.read(NioSocketImpl.java:346)
	at java.base/sun.nio.ch.NioSocketImpl$1.read(NioSocketImpl.java:796)
	at java.base/java.net.Socket$SocketInputStream.read(Socket.java:1099)
	at java.base/java.io.BufferedInputStream.fill(BufferedInputStream.java:291)
	at java.base/java.io.BufferedInputStream.read1(BufferedInputStream.java:347)
	at java.base/java.io.BufferedInputStream.implRead(BufferedInputStream.java:420)
	at java.base/java.io.BufferedInputStream.read(BufferedInputStream.java:399)
	at java.base/java.io.DataInputStream.readFully(DataInputStream.java:208)
	at java.base/ja

# Product Category

In [9]:
cleansed_productcategory_df = productcategory_df.select(
    *[
        col(source_col).alias(target_col).cast(target_data_type)
        for source_col, (target_col, target_data_type) in productcategory_mapping.items()
    ]
)

for source_col, (target_col, target_data_type) in productcategory_mapping.items():
    if target_data_type == "string":
        cleansed_productcategory_df = cleansed_productcategory_df.withColumn(target_col, trim(initcap(col(target_col))))
        

cleansed_productcategory_df.show()


+-----------------+-------------------+------------------+
|ProductCategoryID|ProductCategoryName|ProductSubCategory|
+-----------------+-------------------+------------------+
|                1|              Bikes|    Mountain Bikes|
|                2|              Bikes|        Road Bikes|
|                3|              Bikes|     Touring Bikes|
|                4|         Components|        Handlebars|
|                5|         Components|   Bottom Brackets|
|                6|         Components|            Brakes|
|                7|         Components|            Chains|
|                8|         Components|         Cranksets|
|                9|         Components|       Derailleurs|
|               10|         Components|             Forks|
|               11|         Components|          Headsets|
|               12|         Components|   Mountain Frames|
|               13|         Components|            Pedals|
|               14|         Components|       Road Frame

# Product

In [10]:
cleansed_product_df = product_df.select(
    *[
        col(source_col).alias(target_col).cast(target_data_type)
        for source_col, (target_col, target_data_type) in product_mapping.items()
    ]
)

for source_col, (target_col, target_data_type) in product_mapping.items():
    if target_data_type == "string":
        cleansed_product_df = cleansed_product_df.withColumn(target_col, trim(initcap(col(target_col))))
    if target_data_type == "double":
        cleansed_product_df = cleansed_product_df.withColumn(target_col, round(col(target_col), 2))
    cleansed_product_df = cleansed_product_df.fillna("None")
cleansed_product_df = cleansed_product_df.withColumn("ProductWeightKilograms", when(col("ProductWeightKilograms") == 0, "None").otherwise(col("ProductWeightKilograms")))
cleansed_product_df = cleansed_product_df.withColumn("ProductWeightGrams", (col("ProductWeightKilograms")*1000).cast("int"))
cleansed_product_df = cleansed_product_df.withColumn("ProductProfitAtListPrice", (col("ProductListPrice") - col("ProductCost")).cast("double"))
cleansed_product_df = cleansed_product_df.withColumn("ProductMarginAtListPrice", (col("ProductProfitAtListPrice") / col("ProductCost")).cast("double"))
cleansed_product_df = cleansed_product_df.withColumn("ProductProfitAtListPrice", round(col("ProductProfitAtListPrice"), 2))
cleansed_product_df = cleansed_product_df.withColumn("ProductMarginAtListPrice", round(col("ProductMarginAtListPrice"), 2))

cleansed_product_df.select('ProductCost').show()
    

+-----------+
|ProductCost|
+-----------+
|    1059.31|
|    1059.31|
|      13.09|
|      13.09|
|        3.4|
|        3.4|
|      13.09|
|       6.92|
|      38.49|
|      38.49|
|      38.49|
|      38.49|
|     868.63|
|     868.63|
|     868.63|
|     868.63|
|     868.63|
|     204.63|
|     204.63|
|     204.63|
+-----------+
only showing top 20 rows



# Sales

In [11]:
cleansed_sales_df = sales_df.select(
    *[
        col(source_col).alias(target_col).cast(target_data_type)
        for source_col, (target_col, target_data_type) in sales_mapping.items()
    ]
)

for source_col, (target_col, target_data_type) in sales_mapping.items():
    if target_data_type == "string":
        cleansed_sales_df = cleansed_sales_df.withColumn(target_col, trim(initcap(col(target_col))))
    if target_data_type == "double":
        cleansed_sales_df = cleansed_sales_df.withColumn(target_col, round(col(target_col), 2))
cleansed_sales_df = cleansed_sales_df.withColumn("UnitDiscountValue", (col("UnitPrice") * col("UnitPriceDiscountPercentage")).cast("double"))
cleansed_sales_df = cleansed_sales_df.withColumn("UnitPriceAfterDiscount", (col("UnitPrice") - col("UnitDiscountValue")).cast("double"))
cleansed_sales_df = cleansed_sales_df.withColumn("UnitDiscountValue", round(col("UnitDiscountValue"), 2))
cleansed_sales_df = cleansed_sales_df.withColumn("UnitPriceAfterDiscount", round(col("UnitPriceAfterDiscount"), 2))
cleansed_sales_df = cleansed_sales_df.fillna(0, subset=["UnitPriceDiscountPercentage"])

cleansed_sales_df.show()

+------------+----------------+----------------+----------------+---------+--------------+----------------+-----------------------+----------------+--------------------+---------------------+-------------------+----------+---------+---------------------------+--------------+-----------------+----------------------+
|SalesOrderID|SalesOrderLineID|SalesOrderNumber|  SalesOrderDate|ProductID|SpecialOfferID|SpecialOfferName|SpecialOfferDiscountPct|SpecialOfferType|SpecialOfferCategory|SpecialOfferStartDate|SpecialOfferEndDate|TotalUnits|UnitPrice|UnitPriceDiscountPercentage|TotalLineValue|UnitDiscountValue|UnitPriceAfterDiscount|
+------------+----------------+----------------+----------------+---------+--------------+----------------+-----------------------+----------------+--------------------+---------------------+-------------------+----------+---------+---------------------------+--------------+-----------------+----------------------+
|       45266|            5717|            5717|0

# Source

In [31]:
def read_csv_to_df(file_path, schema):
    df = spark.read.csv(file_path, header="True", schema=schema)
    return df

def productcategory_transformations(file_path, schema):
    df = read_csv_to_df(file_path, schema)

    string_columns = [col_name for col_name, col_type in df.dtypes if col_type == "string"]
    for col_name in string_columns:
        df = df.withColumn(col_name, regexp_replace(col_name, "\\t", " "))
        df = df.withColumn(col_name, trim(initcap(col(col_name))))
    return df


def product_transformations(file_path, schema):
    df = read_csv_to_df(file_path, schema)
    
    string_columns = [col_name for col_name, col_type in df.dtypes if col_type == "string"]
    for col_name in string_columns:
        df = df.withColumn(col_name, trim(col(col_name)))
    double_columns = [col_name for col_name, col_type in df.dtypes if col_type == "double"]
    for col_name in double_columns: 
        df = df.withColumn(col_name, round(col(col_name), 2)).fillna("None")
    df = df.withColumn("ProductWeightKilograms", when(col("ProductWeightKilograms") == 0, None).otherwise(col("ProductWeightKilograms")))
    df = df.withColumn("ProductWeightGrams", (col("ProductWeightKilograms")*1000).cast("int"))
    df = df.withColumn("ProductProfitAtListPrice", (col("ProductListPrice") - col("ProductCost")).cast("double"))
    df = df.withColumn("ProductMarginAtListPrice", (col("ProductProfitAtListPrice") / col("ProductCost")).cast("double"))
    df = df.withColumn("ProductProfitAtListPrice", round(col("ProductProfitAtListPrice"), 2))
    df = df.withColumn("ProductMarginAtListPrice", round(col("ProductMarginAtListPrice"), 2))
    for column in df.columns:
        df = df.withColumn(column, when(col(column).isNull(), None).otherwise(col(column)))
        df = df.withColumn(column, when(col(column) == lit("NULL"), None).otherwise(col(column)))
    return df


def sales_transformations(file_path, schema):
    df = read_csv_to_df(file_path, schema)

    string_columns = [col_name for col_name, col_type in df.dtypes if col_type == "string"]
    for col_name in string_columns:
        df = df.withColumn(col_name, trim(col(col_name)))
    double_columns = [col_name for col_name, col_type in df.dtypes if col_type == "double"]
    for col_name in double_columns: 
        df = df.withColumn(col_name, round(col(col_name), 2))
    df = df.withColumn("UnitDiscountValue", (col("UnitPrice") * col("UnitPriceDiscountPercentage")).cast("double"))
    df = df.withColumn("UnitPriceAfterDiscount", (col("UnitPrice") - col("UnitDiscountValue")).cast("double"))
    df = df.withColumn("UnitDiscountValue", round(col("UnitDiscountValue"), 2))
    df = df.withColumn("UnitPriceAfterDiscount", round(col("UnitPriceAfterDiscount"), 2))
    df = df.fillna(0, subset=["UnitPriceDiscountPercentage"])
    return df

def transformer(file_path, schema):
    if "ProductCategory" in file_path:
        df = productcategory_transformations(file_path, schema)
    elif "Product" in file_path:
        df = product_transformations(file_path, schema)
    elif "Sales" in file_path:
        df = sales_transformations(file_path, schema)
    else:
        return print("Unknown CSV file.")
    return df
    

def write_to_sink(file_path, schema):
    current_date_df = spark.range(1).select(to_timestamp(current_timestamp()).alias("current_date"))
    year_df = current_date_df.select(year("current_date").alias("submission_year"))
    month_df = current_date_df.select(month("current_date").alias("submission_month"))
    day_df = current_date_df.select(day("current_date").alias("submission_day"))
    hour_df = current_date_df.select(hour("current_date").alias("submission_hour"))
    minute_df = current_date_df.select(minute("current_date").alias("submission_minute"))
    second_df = current_date_df.select(second("current_date").alias("submission_second"))

    submission_year = year_df.first()["submission_year"]
    submission_month = month_df.first()["submission_month"]
    submission_day = day_df.first()["submission_day"]
    submission_hour = hour_df.first()["submission_hour"]
    submission_minute = minute_df.first()["submission_minute"]
    submission_second = second_df.first()["submission_second"]

    df = transformer(file_path, schema)
    data_feed = df.withColumn("sourcefile",input_file_name())
    data_feed = data_feed.withColumn("sourcefile", substring_index("sourcefile","/", -1))
    data_feed = data_feed.withColumn("sourcefile", split(col("sourcefile"), "\\.")[0]).select("sourcefile").distinct()
    data_feed = data_feed.withColumn("sourcefile", split(col("sourcefile"), "%")[0]).select("sourcefile").distinct()
    data_feed = data_feed.first()["sourcefile"]

    cleansed_path = f"C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Cleansed/DataFeed={data_feed}/schemaVersion=1/SubmissionYear={submission_year}/SubmissionMonth={submission_month}/SubmissionDay={submission_day}/SubmissionHour={submission_hour}/SubmissionMinute={submission_minute}/SubmissionSecond={submission_second}"
    
    df.write.mode("overwrite").parquet(cleansed_path)
    
    
product_transformations("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Sourced/SystemA/Product", product_schema).show()


+---------------+--------------------+--------------------+------------+-----------+----------------------+----------+-----------------+-----------+----------------+-----------+------------+------------+---------------+--------------+-----------------------+---------------------+-----------------------+------------+--------+------------------+------------------------+------------------------+
|SourceProductID|    ProductModelName|         ProductName|ProductColor|ProductSize|ProductWeightKilograms|ProductUID|ProductCategoryID|ProductCost|ProductListPrice|ProductLine|ProductClass|ProductStyle|ProductMakeFlag|IsFinishedGood|ProductSellingStartDate|ProductSellingEndDate|ProductDiscontinuedDate|ModifiedDate|    date|ProductWeightGrams|ProductProfitAtListPrice|ProductMarginAtListPrice|
+---------------+--------------------+--------------------+------------+-----------+----------------------+----------+-----------------+-----------+----------------+-----------+------------+------------+-----

In [5]:
product = product_transformations("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Sourced/SystemA/Product/date=20220606/Product.csv")
product.select("ProductSellingStartDate", "ProductSellingEndDate", "ProductDiscontinuedDate").distinct().show()


+-----------------------+---------------------+-----------------------+
|ProductSellingStartDate|ProductSellingEndDate|ProductDiscontinuedDate|
+-----------------------+---------------------+-----------------------+
|    31-05-2011 00:00:00|                 Null|                   Null|
|    31-05-2011 00:00:00|  29-05-2013 00:00:00|                   Null|
|    31-05-2011 00:00:00|  29-05-2012 00:00:00|                   Null|
|    30-05-2012 00:00:00|  29-05-2013 00:00:00|                   Null|
|    30-05-2013 00:00:00|                 Null|                   Null|
|    30-04-2008 00:00:00|                 Null|                   Null|
|    30-05-2012 00:00:00|                 Null|                   Null|
+-----------------------+---------------------+-----------------------+



# Target

In [20]:
DimProduct = StructType([
    StructField("ProductKey", LongType()),
    StructField("ProductName", StringType()),
    StructField("ProductLineName", StringType()),
    StructField("ProductModelName", StringType()),
    StructField("ProductCategoryName", StringType()),
    StructField("ProductSubCategoryName", StringType()),   
    StructField("ProductColor", StringType()),
    StructField("ProductSize", StringType()),
    StructField("ProductWeightKilograms", DoubleType()),
    StructField("ProductCost", DoubleType()),
    StructField("ProductListPrice", DoubleType()),
    StructField("ProductProfitAtListPrice", DoubleType()),
    StructField("ProductMarginAtListPrice", DoubleType()),
    StructField("ProductLine", StringType()),
    StructField("ProductClass", StringType()),
    StructField("ProductStyle", StringType()),
    StructField("IsFinishedGood", BooleanType()),
    StructField("ProductSellingStartDate", IntegerType()),
    StructField("ProductSellingEndDate", IntegerType()),
    StructField("ProductDiscontinuedDate", IntegerType()),
    StructField("CurrencyKey", StringType())
])

FactSales = StructType([
    StructField("FactHashID", LongType()),
    StructField("SalesOrderNumber", StringType()),
    StructField("SalesOrderDate", IntegerType()),
    StructField("ProductKey", LongType()),
    StructField("SpecialOfferKey", LongType()),
    StructField("UnitVolume", IntegerType()),
    StructField("CurrencyKey", StringType()),
    StructField("UnitPrice", DoubleType()),
    StructField("UnitPriceDiscountPercentage", DoubleType()),
    StructField("UnitPriceAfterDiscount", DoubleType()),
    StructField("TotalSalesLineValue", DoubleType())
])

In [38]:
#product_df = product_transformations("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Cleansed/DataFeed=Product/date=20220606/Product.csv")
#productcategory_df = productcategory_transformations("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Sourced/SystemA/ProductCategory/date=20220606/ProductCategory.csv")
#sales_df = sales_transformations("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Sourced/SystemA/Sales")

product_df = spark.read.parquet("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Cleansed/DataFeed=Product/schemaVersion=1/SubmissionYear=2023/SubmissionMonth=10/SubmissionDay=30")
productcategory_df = spark.read.parquet("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Cleansed/DataFeed=ProductCategory/schemaVersion=1/SubmissionYear=2023/SubmissionMonth=10/SubmissionDay=30")


pandas_product = product_df.toPandas()
pandas_sales = sales_df.toPandas()
pandas_sales.head()
#sales_df.printSchema()

,SalesOrderDetailID,SalesOrderID,SalesOrderNumber,OrderDate,ProductID,SpecialOfferID,SpecialOfferName,SpecialOfferDiscountPct,SpecialOfferType,SpecialOfferCategory,SpecialOfferStartDate,SpecialOfferEndDate,OrderQty,UnitPrice,UnitPriceDiscount,LineTotal
0,5717,45266,SO45266,01/01/2012 00:00,777,1,No Discount,0,No Discount,No Discount,01/05/2011 00:00,30/11/2014 00:00,2,2024.994,0,4049.988
1,5718,45266,SO45266,01/01/2012 00:00,774,1,No Discount,0,No Discount,No Discount,01/05/2011 00:00,30/11/2014 00:00,3,2039.994,0,6119.982
2,5719,45266,SO45266,01/01/2012 00:00,775,1,No Discount,0,No Discount,No Discount,01/05/2011 00:00,30/11/2014 00:00,2,2024.994,0,4049.988
3,5720,45266,SO45266,01/01/2012 00:00,743,1,No Discount,0,No Discount,No Discount,01/05/2011 00:00,30/11/2014 00:00,1,714.7043,0,714.7043
4,5721,45266,SO45266,01/01/2012 00:00,778,1,No Discount,0,No Discount,No Discount,01/05/2011 00:00,30/11/2014 00:00,2,2024.994,0,4049.988


In [40]:
product = product_df
date_columns = ["ProductSellingStartDate", "ProductSellingEndDate", "ProductDiscontinuedDate"]
for column in date_columns:
    product = product.withColumn(column, date_format(to_date(col(column), "dd-MM-yyyy HH:mm:ss"), "yyyyMMdd").cast("int"))

product = product.withColumn("CurrencyKey", lit("GBP"))
product = product.withColumn("ProductLineName", when(col("ProductLine") == "M", "Mountain Bikes")
                                                .when(col("ProductLine") == "Null", "Bike Parts")
                                                .when(col("ProductLine") == "R", "Road Bikes")
                                                .when(col("ProductLine") == "S", "Accessories and Attire")
                                                .when(col("ProductLine") == "T", "Touring Bikes")
                                                .otherwise("Unknown"))
product = product.withColumn("ProductKey", xxhash64(concat_ws("|", col("SourceProductID"), lit("Product"), lit("ProductCategory"))).cast("long"))
product = product.join(productcategory_df, on="ProductCategoryID", how="left")
product = product.withColumnRenamed("ProductSubCategory", "ProductSubCategoryName")
for col_name in product.columns:
    data_type = product.schema[col_name].dataType
    if data_type == DoubleType:
        product = product.withColumn(col_name, round(col(col_name), 2))
product = product.drop("SourceProductID", "ProductUID", "ProductCategoryID", "ProductMakeFlag", "ModifiedDate", "ProductWeightGrams")

product.printSchema()

root
 |-- ProductModelName: string (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- ProductColor: string (nullable = true)
 |-- ProductSize: string (nullable = true)
 |-- ProductWeightKilograms: double (nullable = true)
 |-- ProductCost: double (nullable = true)
 |-- ProductListPrice: double (nullable = true)
 |-- ProductLine: string (nullable = true)
 |-- ProductClass: string (nullable = true)
 |-- ProductStyle: string (nullable = true)
 |-- IsFinishedGood: boolean (nullable = true)
 |-- ProductSellingStartDate: integer (nullable = true)
 |-- ProductSellingEndDate: integer (nullable = true)
 |-- ProductDiscontinuedDate: integer (nullable = true)
 |-- date: integer (nullable = true)
 |-- ProductProfitAtListPrice: double (nullable = true)
 |-- ProductMarginAtListPrice: double (nullable = true)
 |-- SubmissionHour: integer (nullable = true)
 |-- SubmissionMinute: integer (nullable = true)
 |-- SubmissionSecond: integer (nullable = true)
 |-- CurrencyKey: string (nullable

In [54]:
sales = sales_df
date_columns = ["SalesOrderDate"]
for column in date_columns:
    sales = sales.withColumn(column, date_format(to_date(col(column), "dd/MM/yyyy HH:mm"), "yyyyMMdd").cast("int"))
    
FactHashID_columns = sales.columns
sales = sales.withColumn("FactHashID", xxhash64(concat_ws("|", *[col(column) for column in FactHashID_columns])).cast("long"))
sales = sales.withColumn("CurrencyKey", lit("GBP"))
sales = sales.withColumn("ProductKey", xxhash64(concat_ws("|", col("ProductID"), lit("Product"), lit("ProductCategory"))).cast("long"))
#FactHashID_columns = ["SalesOrderNumber", "SalesOrderDate", "TotalUnits", "UnitPrice", "UnitPriceDiscountPercentage", "UnitPriceAfterDiscount", "TotalLineValue"]
#sales = sales.withColumn("FactHashID", xxhash64(concat_ws("|", *[col(column) for column in FactHashID_columns])).cast("long"))
sales = sales.withColumn("SpecialOfferKey", xxhash64(concat_ws("|", col("SpecialOfferID"), lit("Sales"))).cast("long"))
sales = sales.drop("SalesOrderID", "SalesOrderLineID", "ProductID", "SpecialOfferID", "SpecialOfferName", "SpecialOfferDiscountPct", "SpecialOfferType", "SpecialOfferCategory", "SpecialOfferStartDate", "SpecialOfferEndDate", "UnitDiscountValue")
sales = sales.withColumnRenamed("TotalUnits", "UnitVolume").withColumnRenamed("TotalLineValue", "TotalSalesLineValue")
#Partition the file by Month and Year from the field SalesOrderDate

sales.show()

+----------------+--------------+----------+---------+---------------------------+-------------------+----------------------+--------------------+-----------+--------------------+-------------------+
|SalesOrderNumber|SalesOrderDate|UnitVolume|UnitPrice|UnitPriceDiscountPercentage|TotalSalesLineValue|UnitPriceAfterDiscount|          FactHashID|CurrencyKey|          ProductKey|    SpecialOfferKey|
+----------------+--------------+----------+---------+---------------------------+-------------------+----------------------+--------------------+-----------+--------------------+-------------------+
|         SO45266|      20120101|         2|  2024.99|                        0.0|            4049.99|               2024.99| -567102246171116547|        GBP|-7526531161666529242|7804943415359386941|
|         SO45266|      20120101|         3|  2039.99|                        0.0|            6119.98|               2039.99| 2099134126650911946|        GBP|-5247391021351068957|7804943415359386941|


In [87]:
current_date_df = spark.range(1).select(to_timestamp(current_timestamp()).alias("current_date"))
year_df = current_date_df.select(year("current_date").alias("submission_year"))
month_df = current_date_df.select(month("current_date").alias("submission_month"))
day_df = current_date_df.select(day("current_date").alias("submission_day"))
hour_df = current_date_df.select(hour("current_date").alias("submission_hour"))
minute_df = current_date_df.select(minute("current_date").alias("submission_minute"))
second_df = current_date_df.select(second("current_date").alias("submission_second"))

submission_year = year_df.first()["submission_year"]
submission_month = month_df.first()["submission_month"]
submission_day = day_df.first()["submission_day"]
submission_hour = hour_df.first()["submission_hour"]
submission_minute = minute_df.first()["submission_minute"]
submission_second = second_df.first()["submission_second"]

df = spark.read.csv("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Sourced/SystemA/Product", header="True")
entity = df.withColumn("sourcefile",input_file_name())
entity = entity.withColumn("sourcefile", substring_index("sourcefile","/", -1))
entity = entity.withColumn("sourcefile", split(col("sourcefile"), "\\.")[0]).select("sourcefile").distinct()
entity = entity.withColumn("sourcefile", split(col("sourcefile"), "%")[0]).select("sourcefile").distinct()
entity = entity.first()["sourcefile"]
if entity == "Product":
    domain = "Master"
elif entity == "Sales":
    domain = "Commercial"

integrated_path = f"C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Integrated/Domain={domain}/Entity={entity}/schemaVersion=1/SubmissionYear={submission_year}/SubmissionMonth={submission_month}/SubmissionDay={submission_day}/SubmissionHour={submission_hour}/SubmissionMinute={submission_minute}/SubmissionSecond={submission_second}"
print(integrated_path)

C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Integrated/Domain=Master/Entity=Product/schemaVersion=1/SubmissionYear=2023/SubmissionMonth=10/SubmissionDay=18/SubmissionHour=16/SubmissionMinute=34/SubmissionSecond=20


In [29]:
def read_parquet_to_df(file_path):
    df = spark.read.parquet(file_path, header="True")
    return df
    
def read_latest_parquet(file_path):
    latest_file = None
    latest_timestamp = 0

    for root, dirs, files in os.walk(file_path):
        for file in files:
            file_timestamp = os.path.getctime(root)

            if file_timestamp > latest_timestamp:
                latest_timestamp = file_timestamp
                latest_file = root
            
    if latest_file:
        df = spark.read.parquet(latest_file)
        return df


def product_integrated(file_path):
    df = read_parquet_to_df(file_path)
    ProductCategory_df = read_latest_parquet("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Cleansed/DataFeed=ProductCategory")
    DimProduct_df = df
    date_columns = ["ProductSellingStartDate", "ProductSellingEndDate", "ProductDiscontinuedDate"]
    for column in date_columns:
        DimProduct_df = DimProduct_df.withColumn(column, date_format(to_date(col(column), "dd-MM-yyyy HH:mm:ss"), "yyyyMMdd").cast("int"))

    DimProduct_df = DimProduct_df.withColumn("CurrencyKey", lit("GBP"))
    DimProduct_df = DimProduct_df.withColumn("ProductLineName", when(col("ProductLine") == "M", "Mountain Bikes")
                                                                .when(col("ProductLine") == "Null", "Bike Parts")
                                                                .when(col("ProductLine") == "R", "Road Bikes")
                                                                .when(col("ProductLine") == "S", "Accessories and Attire")
                                                                .when(col("ProductLine") == "T", "Touring Bikes")
                                                                .otherwise("Unknown"))
    DimProduct_df = DimProduct_df.withColumn("ProductKey", xxhash64(concat_ws("|", col("SourceProductID"), lit("Product"), lit("ProductCategory"))).cast("long"))
    DimProduct_df = DimProduct_df.join(ProductCategory_df, on="ProductCategoryID", how="left")
    DimProduct_df = DimProduct_df.withColumnRenamed("ProductSubCategory", "ProductSubCategoryName")
    for col_name in DimProduct_df.columns:
        data_type = DimProduct_df.schema[col_name].dataType
        DimProduct_df = DimProduct_df.withColumn(col_name, col(col_name).cast(data_type))
        if data_type == DoubleType:
            DimProduct_df = DimProduct_df.withColumn(col_name, round(col(col_name), 2))
    DimProduct_df = DimProduct_df.select(DimProduct.fieldNames())
    return DimProduct_df


def sales_integrated(file_path):
    df = read_parquet_to_df(file_path)

    FactSales_df = df
    date_columns = ["SalesOrderDate"]
    for column in date_columns:
        FactSales_df = FactSales_df.withColumn(column, date_format(to_date(to_timestamp(col(column), "dd/MM/yyyy HH:mm")), "yyyyMMdd").cast("int"))

    FactSales_df = FactSales_df.withColumn("CurrencyKey", lit("GBP"))
    FactSales_df = FactSales_df.withColumn("ProductKey", xxhash64(concat_ws("|", col("ProductID"), lit("Product"), lit("ProductCategory"))).cast("long"))
    FactHashID_columns = ["SalesOrderNumber", "SalesOrderDate", "TotalUnits", "UnitPrice", "UnitPriceDiscountPercentage", "UnitPriceAfterDiscount", "TotalLineValue"]
    FactSales_df = FactSales_df.withColumn("FactHashID", xxhash64(concat_ws("|", *[col(column) for column in FactHashID_columns])).cast("long"))
    FactSales_df = FactSales_df.withColumn("SpecialOfferKey", xxhash64(concat_ws("|", col("SpecialOfferID"), lit("Sales"))).cast("long"))
    FactSales_df = FactSales_df.withColumnRenamed("TotalUnits", "UnitVolume").withColumnRenamed("TotalLineValue", "TotalSalesLineValue")
    FactSales_df = FactSales_df.select(FactSales.fieldNames())
    #Partition the file by Month and Year from the field SalesOrderDate
    return df

def transformer(file_path):
    if "Product" in file_path:
        df = product_integrated(file_path)
    elif "Sales" in file_path:
        df = sales_integrated(file_path)
    else:
        return print("Unknown parquet file.")
    return df

def write_to_sink(file_path):
    current_date_df = spark.range(1).select(to_timestamp(current_timestamp()).alias("current_date"))
    year_df = current_date_df.select(year("current_date").alias("submission_year"))
    month_df = current_date_df.select(month("current_date").alias("submission_month"))
    day_df = current_date_df.select(day("current_date").alias("submission_day"))
    hour_df = current_date_df.select(hour("current_date").alias("submission_hour"))
    minute_df = current_date_df.select(minute("current_date").alias("submission_minute"))
    second_df = current_date_df.select(second("current_date").alias("submission_second"))

    submission_year = year_df.first()["submission_year"]
    submission_month = month_df.first()["submission_month"]
    submission_day = day_df.first()["submission_day"]
    submission_hour = hour_df.first()["submission_hour"]
    submission_minute = minute_df.first()["submission_minute"]
    submission_second = second_df.first()["submission_second"]

    df = transformer(file_path)
    if "Product" in file_path:
        entity = "Product"
        domain = "Master"
    elif "Sales" in file_path:
        entity = "Sales"
        domain = "Commercial"    
    
    integrated_path = f"C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Integrated/Domain={domain}/Entity={entity}/schemaVersion=1/SubmissionYear={submission_year}/SubmissionMonth={submission_month}/SubmissionDay={submission_day}/SubmissionHour={submission_hour}/SubmissionMinute={submission_minute}/SubmissionSecond={submission_second}"


    if entity == "Product":
        df.write.mode("overwrite").parquet(integrated_path)
    elif entity == "Sales":
        df = df.withColumn("SalesOrderYear", substring("SalesOrderDate", 1, 4)).withColumn("SalesOrderMonth", substring("SalesOrderDate", 5, 2))
        df.write.partitionBy("SalesOrderYear", "SalesOrderMonth").mode("overwrite").parquet(integrated_path)
    else:
        return print("Unknown parquet file.")
        
transformer("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Cleansed/DataFeed=Sales/schemaVersion=1/SubmissionYear=2023/SubmissionMonth=10/SubmissionDay=20/SubmissionHour=12").printSchema()

root
 |-- SalesOrderID: string (nullable = true)
 |-- SalesOrderLineID: string (nullable = true)
 |-- SalesOrderNumber: string (nullable = true)
 |-- SalesOrderDate: string (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- SpecialOfferID: string (nullable = true)
 |-- SpecialOfferName: string (nullable = true)
 |-- SpecialOfferDiscountPct: string (nullable = true)
 |-- SpecialOfferType: string (nullable = true)
 |-- SpecialOfferCategory: string (nullable = true)
 |-- SpecialOfferStartDate: string (nullable = true)
 |-- SpecialOfferEndDate: string (nullable = true)
 |-- TotalUnits: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- UnitPriceDiscountPercentage: double (nullable = true)
 |-- TotalLineValue: double (nullable = true)
 |-- UnitDiscountValue: double (nullable = true)
 |-- UnitPriceAfterDiscount: double (nullable = true)
 |-- SubmissionMinute: integer (nullable = true)
 |-- SubmissionSecond: integer (nullable = true)



In [12]:
import os
import glob
import pandas as pd

def read_latest_parquet(file_path):
    latest_file = None
    latest_timestamp = 0

    for root, dirs, files in os.walk(file_path):
        for file in files:
            file_timestamp = os.path.getctime(root)

            if file_timestamp > latest_timestamp:
                latest_timestamp = file_timestamp
                latest_file = root
            
    if latest_file:
        df = spark.read.parquet(latest_file)
        return df

read_latest_parquet('C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Cleansed/DataFeed=ProductCategory/').show()


+-----------------+-------------------+------------------+
|ProductCategoryID|ProductCategoryName|ProductSubCategory|
+-----------------+-------------------+------------------+
|                1|              Bikes|    Mountain Bikes|
|                2|              Bikes|        Road Bikes|
|                3|              Bikes|     Touring Bikes|
|                4|         Components|        Handlebars|
|                5|         Components|   Bottom Brackets|
|                6|         Components|            Brakes|
|                7|         Components|            Chains|
|                8|         Components|         Cranksets|
|                9|         Components|       Derailleurs|
|               10|         Components|             Forks|
|               11|         Components|          Headsets|
|               12|         Components|   Mountain Frames|
|               13|         Components|            Pedals|
|               14|         Components|       Road Frame

In [29]:
df = spark.read.csv("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Sourced/SystemA/Sales")
df.show()

+------------------+------------+----------------+----------------+---------+--------------+----------------+--------------------+----------------+--------------------+--------------------+-------------------+--------+---------+-----------------+---------+--------+
|               _c0|         _c1|             _c2|             _c3|      _c4|           _c5|             _c6|                 _c7|             _c8|                 _c9|                _c10|               _c11|    _c12|     _c13|             _c14|     _c15|    date|
+------------------+------------+----------------+----------------+---------+--------------+----------------+--------------------+----------------+--------------------+--------------------+-------------------+--------+---------+-----------------+---------+--------+
|SalesOrderDetailID|SalesOrderID|SalesOrderNumber|       OrderDate|ProductID|SpecialOfferID|SpecialOfferName|SpecialOfferDisco...|SpecialOfferType|SpecialOfferCategory|SpecialOfferStart...|SpecialOfferE

In [28]:
import pandas as pd

data = [
    ["1", "   bikes", "mountain bikes"],
    ["2", "   bikes", "road bikes "],
    ["3", "Bikes", "touring bikes"],
    ["4", "components  ", "handlebars"],
    ["5", "components	", "bottom brackets"],
    ["6", "components	", "brakes"],
    ["7", "components	", "chains"],
    ["8", "components	", "cranksets"],
    ["9", "components", "derailleurs"],
    ["10", "components", "forks"],
    ["11", "components", "headsets"],
    ["12", "components", "mountain frames"],
    ["13", "components", "pedals"],
    ["14", "components", "road frames"],
    ["15", "components", "saddles"],
    ["16", "components", "touring frames"],
    ["17", "components", "wheels"],
    ["18", "clothing", "Bib-shorts"],
    ["19", "clothing", "CAPS"],
    ["20", "clothing", "GLOVES"],
    ["21", " clothing", "JERSEYS"],
    ["22", "clothing  ", "SHORTS"],
    ["23", "clothing  ", "SOCKS"],
    ["24", "clothing", "helmet"]
]

columns = ["ID", "Level1", "Level2"]

df = pd.DataFrame(data, columns=columns)

print(df)

    ID        Level1           Level2
0    1         bikes   mountain bikes
1    2         bikes      road bikes 
2    3         Bikes    touring bikes
3    4  components         handlebars
4    5  components\t  bottom brackets
5    6  components\t           brakes
6    7  components\t           chains
7    8  components\t        cranksets
8    9    components      derailleurs
9   10    components            forks
10  11    components         headsets
11  12    components  mountain frames
12  13    components           pedals
13  14    components      road frames
14  15    components          saddles
15  16    components   touring frames
16  17    components           wheels
17  18      clothing       Bib-shorts
18  19      clothing             CAPS
19  20      clothing           GLOVES
20  21      clothing          JERSEYS
21  22    clothing             SHORTS
22  23    clothing              SOCKS
23  24      clothing           helmet


In [22]:
def find_schema(schema):
    if schema in locals():
        return locals()[schema]
    else:
        return None
    
print(find_schema(DimProduct))

None


In [11]:
df = spark.read.parquet("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Integrated/Domain=Commercial/Entity=Sales/schemaVersion=1/SubmissionYear=2023/SubmissionMonth=10/SubmissionDay=31/SubmissionHour=10/SubmissionMinute=10")
df.select("SalesOrderDate").distinct().show()

df = spark.read.parquet("C:/Users/TobyBarker/Documents/Data_Engineering_Accelerator/src/databricks/sample_lake/Cleansed/DataFeed=Sales/schemaVersion=1/SubmissionYear=2023/SubmissionMonth=10/SubmissionDay=30")
df.select("SalesOrderDate").distinct().show()

+--------------+
|SalesOrderDate|
+--------------+
|      20140314|
|      20140515|
|      20140501|
|      20140519|
|      20130703|
|      20130728|
|      20140312|
|      20130707|
|      20140529|
|      20130726|
|      20140309|
|      20140303|
|      20140320|
|      20140505|
|      20130722|
|      20140310|
|      20140318|
|      20140328|
|      20140506|
|      20140512|
+--------------+
only showing top 20 rows

+----------------+
|  SalesOrderDate|
+----------------+
|17/06/2014 00:00|
|17/05/2014 00:00|
|14/06/2014 00:00|
|19/04/2014 00:00|
|24/03/2014 00:00|
|24/06/2014 00:00|
|29/01/2014 00:00|
|20/02/2014 00:00|
|03/04/2014 00:00|
|02/06/2014 00:00|
|25/05/2014 00:00|
|06/06/2014 00:00|
|18/02/2014 00:00|
|09/05/2014 00:00|
|23/02/2014 00:00|
|10/04/2014 00:00|
|17/03/2014 00:00|
|03/03/2014 00:00|
|21/04/2014 00:00|
|07/06/2014 00:00|
+----------------+
only showing top 20 rows

